# CoT (Chain of Thought)

In [1]:
from dotenv import load_dotenv
import openai

load_dotenv()
client = openai.OpenAI()

### 일반 프롬프트 vs Zero-shot CoT

In [7]:
# 일반 프롬프트
prompt = '123 x 31 + 0.123 x 0.0012는 얼마인가요?'

response = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content': prompt}]
)

In [8]:
print(response.choices[0].message.content)

3813.0000001476 입니다.


In [9]:
# CoT 프롬프트
prompt = '단계별로 계산하세요.: 123 x 31 + 0.123 x 0.0012는 얼마인가요?'

cot_response = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content': prompt}]
)

In [10]:
print(cot_response.choices[0].message.content)

1단계: 123 x 31 = 3813
2단계: 0.123 x 0.0012 = 0.0001476
3단계: 3813 + 0.0001476 = 3813.0001476

따라서, 123 x 31 + 0.123 x 0.0012 = 3813.0001476입니다.


### Few-shot CoT

In [11]:
# 일반 프롬프트
prompt = '123 x 31은 얼마인가요?'

response = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content': prompt}]
)

In [12]:
print(response.choices[0].message.content)

123 x 31 = 3813


In [15]:
# Few-shot CoT
few_shot_cot = '''
Q: 123 x 31은 얼마인가요?
A: 
1. 123을 분해합니다: 123 = 100 + 20 + 3
2. 각 항을 31과 곱합니다:
    - 100 x 31 = 3100
    - 20 x 31 = 620
    - 3 x 31 = 93
3. 이제 이 결과들을 더합니다:
    - 3100 + 620 + 93
4. 계산을 진행하면:
    - 3100 + 620 = 3720
    - 3720 + 93 = 3813
따라서 123 x 31 = 3813 입니다.

Q: 789 x 56 은 얼마인가요?
A: 
'''

cot_response = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content': few_shot_cot}]
)

In [16]:
print(cot_response.choices[0].message.content)

1. 789을 분해합니다: 789 = 700 + 80 + 9
2. 각 항을 56과 곱합니다:
    - 700 x 56 = 39200
    - 80 x 56 = 4480
    - 9 x 56 = 504
3. 이제 이 결과들을 더합니다:
    - 39200 + 4480 + 504
4. 계산을 진행하면:
    - 39200 + 4480 = 43680
    - 43680 + 504 = 44184
따라서 789 x 56 = 44184 입니다.


### 논리 문제 해결

In [31]:
prompt = '''
단계적으로 생각해서 대답해 주세요.:
앨리스(Alice)는 밥(Bob)보다 나이가 많다.
밥(Bob)은 찰리(Charlie)보다 나이가 많다.
데이비드(David)는 앨리스(Alice)보다 어리지만, 에밀리(Emily)보다는 많다.
찰리(Charlie)는 프랭크(Frank)보다 많다.
에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다는 어리다.
프랭크(Frank)는 데이비드(David)보다 어리다.
그렇다면 가장 어린 사람은 누구인가요?
'''

In [32]:
response = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content': prompt}]
)

In [33]:
print(response.choices[0].message.content)

먼저, 첫 번째 세 문장을 통해 앨리스는 밥보다 많고, 밥은 찰리보다 많으므로, 앨리스가 밥, 찰리보다 나이가 많다는 것을 알 수 있습니다. 

두 번째로, 찰리가 프랭크보다 많으므로 프랭크는 앨리스, 밥, 찰리 보다 어린 것이 분명합니다.

세 번째로, 데이비드는 앨리스보다 어리지만, 에밀리보다 나이가 많다고 했습니다. 앨리스는 프랭크보다 많으므로, 데이비드도 프랭크보다는 많습니다.

마지막으로, 에밀리는 찰리보다 많지만 밥보다 어리다고 했으므로 에밀리는 앨리스보다 어리며, 찰리보다 많으므로 프랭크보다도 많습니다.

결론적으로, 가장 어린 사람은 프랭크(Frank)입니다.


이 문제를 해결하려면, 주어진 정보를 순서대로 정리해야 합니다.

1. 앨리스(Alice) > 밥(Bob)
2. 밥(Bob) > 찰리(Charlie)
3. 찰리(Charlie) > 프랭크(Frank)
4. 데이비드(David) > 프랭크(Frank) but 데이비드(David) < 앨리스(Alice)
5. 에밀리(Emily) > 찰리(Charlie) but 에밀리(Emily) < 밥(Bob)

이 정보를 통해 우리는 앨리스가 가장 연장자이고, 밥이 그 다음, 그리고 밥보다 어린 사람들 중에서 에밀리가 찰리보다 나이가 많고, 데이비드와 프랭크는 찰리보다 어리며, 데이비드는 프랭크보다 나이가 많습니다. 따라서 가장 어린 사람은 프랭크입니다.